In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

# Langsmith tracking
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACKING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = os.getenv("LANGCHAIN_PROJECT")

In [ ]:
## Data Ingestion -- From the website we need to scrape the data

from langchain_community.document_loaders import WebBaseLoader

In [ ]:
loader = WebBaseLoader("https://docs.smith.langchain.co/tutorials/Administrators/manage_spend")
loader

In [ ]:
docs = loader.load()
docs

In [ ]:
### Load Data --> Docs ---> Divide our text into chunks --> vectors --> Vector Embeddings

from langchain_text_splitters import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000,chunk_overlay=200)
documents = text_splitter.split_documents(docs)



In [ ]:
documents

In [ ]:
from langchain_open import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()


In [ ]:
from langchain_community.vectorstores import FAISS

vectorstoredb = FAISS.from_documents(documents,embeddings)

In [ ]:
vectorstoredb

In [ ]:
## Query from a vector db

query= "Langsmith has two usage limits : total traces extended"
result  = vectorstoredb.similarity_search(query)
result[0].page_content

In [ ]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o")

In [ ]:
## Retrieval Chain, Document chain

from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(
    """
Answer the following question based only on the provided context
<context>
{context}
</context>
"""
)

document_chain = create_stuff_documents_chain(llm,prompt)
document_chain

In [ ]:
from langchain_core.documents import Document


document_chain.invoke(
    "input","Langsmith has two usage limits : total traces and extended"
    ,"context"
)

In [ ]:
vectorstoredb

In [ ]:
retriever = vectorstoredb.as_retriever()
from langchain.chains import create_retrieval_chain
retrieval_chain = create_retrieval_chain(retriever,document_chain)

In [ ]:
retrieval_chain

In [ ]:
### Get the response from the LLM
response=retrieval_chain.invoke({"input":"LangSmith has two usage limits : total traces and extended"})
response["answer"]

In [ ]:
response['context']